In [1]:
from redis import Redis
REDIS = Redis('this_redis')

# Redis

One interesting use case is to have Redis track the evolution of some long running iterative process.

In [2]:
import time

def some_iterative_process():
    time.sleep(1)

In [3]:
some_iterative_process()

In [4]:
count = 0
REDIS.set('count', count)

True

In [5]:
while count < 30:
    some_iterative_process()
    count = REDIS.incr('count')

In [6]:
count

30

## Passing Parameters Between Processes

We could also use Redis to pass parameters between procesess.

In [7]:
import numpy as np
import json
model_params = {'C':list(np.logspace(-3,3,7)),
                'penalty':'l1',
                'solver' : 'newton-cg'}

In [8]:
model_params

{'C': [0.001, 0.01, 0.10000000000000001, 1.0, 10.0, 100.0, 1000.0],
 'penalty': 'l1',
 'solver': 'newton-cg'}

In [9]:
type(model_params)

dict

In [10]:
json.dumps(model_params)

'{"C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], "penalty": "l1", "solver": "newton-cg"}'

In [11]:
type(json.dumps(model_params))

str

In [12]:
REDIS.set('model_params', json.dumps(model_params))

True

## Passing Arrays Between Processes

We could also use Redis to pass arrays between procesess.

In [13]:
A = np.array([[1,1,1],
              [2,2,2],
              [3,3,3]])

In [15]:
n, m = A.shape
n, m

(3, 3)

In [16]:
encoded_A = A.ravel().tostring()

In [18]:
encoded_A

b'\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00'

In [17]:
REDIS.set('encoded_A', encoded_A)

True

In [23]:
REDIS.set('A_n', n)
REDIS.set('A_m', m)
REDIS.set('A_type', 'int')

True

In [24]:
def matrix_to_redis(matrix, redis_client, key):
    n, m = matrix.shape
    encoded_mat = matrix.ravel().tostring()
    redis_client.set(key, encoded_mat)
    redis_client.set(key+'_n', n)
    redis_client.set(key+'_m', m)

In [25]:
matrix_to_redis(A, REDIS, 'new_A')

This could also be done via the `pickle` library.

In [26]:
import pickle
import numpy as np

B = np.random.randint(1,10,9).reshape(3,3)
B_pkl = pickle.dumps(B)

REDIS.set('B', B_pkl)

True

In [27]:
B

array([[7, 6, 8],
       [5, 5, 1],
       [4, 2, 4]])